### Step 1: *Data Processing*
The goal for this notebook is to prepare Citi Bike data (123 files, 36.79GB ) for analysis.
 

From **June 2013 - January 2021**, Citi Bike data is stored in the following format:

```
'tripduration','starttime','stoptime','start station id','start station name','start station latitude','start station longitude','end station id','end station name','end station latitude','end station longitude','bikeid','usertype','birth year','gender'
```
 

 
From **February 2021 - July 2023**, Citi Bike data is stored in the following format:

```
ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
```
 
 
 
 
After looking at the data, we can notice several differences in how the data is stored:
1. Post February 2021, gender, Bike ID, birth year, and trip duration are not stored. These columns can be dropped.
2. Station ID in pre-February 2021 data is a four-digit integer (3276 for Marin Light Rail). Post-February 2021, the station ID is changed to a combination of characters and integers (JC008 for Marin Light Rail)
3. User Type in pre-February 2021 data is collected as 'Subscriber' or 'Customer'. Post-February 2021, the user type is collected as 'member' or 'casual'. We can convert 'Subscriber' to 'member' and 'Customer' to 'casual' for consistency.
4. The column names are different, these can be renamed to match the post-February 2021 data.
    - we will need to check if the coordinates for each station are consistent across the two date ranges
5. The start and end times are also collected differently. We will need to convert the pre-February 2021 data to match the post-February 2021 data.
 
 While Pandas is a great tool for most data processing tasks, it is limited when it comes to processing large datasets. We'll use Polars to process, clean, merge, and format the data. We'll then save the data by year to a parquet file for analysis in the next notebook. A single parquet file for each year will be much easier to manage than 123 csv files.

In [2]:
import polars as pl
import pandas as pd

In [10]:
'''
First, we'll need to correct for irregular date formatting from 2014-09 to 2021-01.
    1/1/2016 00:04:43 -> 2016-01-01 00:04:43
    2016-01-01 00:04:43.0000 -> 2016-01-01 00:04:43
We'll do this using pandas and rewrite the csv file.
'''

correction_start = pd.to_datetime('201409', format='%Y%m')
correction_end = pd.to_datetime('202101', format='%Y%m')
# create date range
date_range = pd.date_range(start=correction_start, end=correction_end, freq='M')
date_range = [date.strftime('%Y%m') for date in pd.date_range(correction_start, correction_end, freq='MS')]

# Read in the data

for date in date_range:
    df = pd.read_csv(f'./source-data/{date}-citibike-tripdata.csv')
    df['starttime'] = pd.to_datetime(df['starttime']).dt.round(freq='S')
    df['stoptime'] = pd.to_datetime(df['stoptime']).dt.round(freq='S')
    df.to_csv(f'./source-data/{date}-citibike-tripdata.csv', index=False)

In [19]:
'''
Create a date range list for each year
    - first date range is from 201306 to 202101
    - second date range is from 202102 to 202307
lists will be used to read in csv files, and concatenate them into a parquet file for each year
'''
date_ranges_1 = []

# first: date strings for 2013-06 to 2013-12
start_date = pd.to_datetime('201306', format='%Y%m')
end_date = pd.to_datetime('201312', format='%Y%m')
date_ranges_1.append([date.strftime('%Y%m') for date in pd.date_range(start_date, end_date, freq='MS')])

for year in range(2014, 2021):
    start_date = pd.to_datetime(str(year) + '01', format='%Y%m')
    end_date = pd.to_datetime(str(year) + '12', format='%Y%m')
    date_ranges_1.append([date.strftime('%Y%m') for date in pd.date_range(start_date, end_date, freq='MS')])

start_date = pd.to_datetime('202101', format='%Y%m')
end_date = pd.to_datetime('202101', format='%Y%m')
date_ranges_1.append([date.strftime('%Y%m') for date in pd.date_range(start_date, end_date, freq='MS')])

# second: date strings for Feb 2021 - Jul 2023
date_ranges_2 = []
start_date = pd.to_datetime('20210201', format='%Y%m%d')
end_date = pd.to_datetime('20211201', format='%Y%m%d')
date_ranges_2.append([date.strftime('%Y%m') for date in pd.date_range(start_date, end_date, freq='MS')])

start_date = pd.to_datetime('20220101', format='%Y%m%d')
end_date = pd.to_datetime('20221201', format='%Y%m%d')
date_ranges_2.append([date.strftime('%Y%m') for date in pd.date_range(start_date, end_date, freq='MS')])

start_date = pd.to_datetime('20230101', format='%Y%m%d')
end_date = pd.to_datetime('20230701', format='%Y%m%d')
date_ranges_2.append([date.strftime('%Y%m') for date in pd.date_range(start_date, end_date, freq='MS')])

In [20]:
'''
create a function to loop through date ranges to read each csv file and concat into one polars dataframe
'''

datapath = './source-data/{}-citibike-tripdata.csv'

def create_df(date_range, path):
    for date in date_range:
        df = pl.read_csv(path.format(date), infer_schema_length=0)
        if date == date_range[0]:
            df_all = df
        else:
            df_all = pl.concat([df_all, df])
    return df_all


In [21]:
'''
Define columns to be dropped
- for this analysis, we'll exclude birth year and gender columns
'''
columns = ['birth year', 'gender', 'tripduration']

'''
Function to rename columns
- change the column names to maintain consistency across datasets. 
- data format changes Feb 2021, pre Feb 2021 data is changed to match post Feb 2021 data
'''
def rename_cols(df):
    renames = {
            'starttime': 'started_at',
            'stoptime': 'ended_at',
            'start station id': 'start_station_id',
            'start station name': 'start_station_name',
            'start station latitude': 'start_lat',
            'start station longitude': 'start_lng',
            'end station id': 'end_station_id',
            'end station name': 'end_station_name',
            'end station latitude': 'end_lat',
            'end station longitude': 'end_lng',
            'bikeid': 'bike_id',
            'usertype': 'member_casual',
        }
    df = df.rename(renames)
    return df

'''
Function to cast types for each column
'''
def cast_types(df):
    out = df.select(
        pl.col('started_at').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S', strict=False),
        pl.col('ended_at').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S', strict=False),
        pl.col('start_station_id').cast(pl.Int32, strict=False),
        pl.col('start_station_name').cast(pl.Utf8, strict=False),
        pl.col('start_lat').cast(pl.Float32, strict=False),
        pl.col('start_lng').cast(pl.Float32, strict=False),
        pl.col('end_station_id').cast(pl.Int32, strict=False),
        pl.col('end_station_name').cast(pl.Utf8, strict=False),
        pl.col('end_lat').cast(pl.Float32, strict=False),
        pl.col('end_lng').cast(pl.Float32, strict=False),
        pl.col('bike_id').cast(pl.Int32, strict=False),
        pl.col('member_casual').cast(pl.Utf8, strict=False),
    )
    return out

'''
Function to change values for member_casual column
- change 'Subscriber' to 'member' and 'Customer' to 'casual'
'''
def change_member_casual(df):
    map_dict = {
        'Subscriber': 'member',
        'Customer': 'casual',
        None: 'unknown'
    }

    df = df.with_columns(
        pl.col('member_casual').map_dict(map_dict, default='unknown').alias('member_casual')
    )

    return df

In [23]:
for date_range in date_ranges_1:
    filename = date_range[0][:4] + '.parquet'
    df = create_df(date_range, datapath)
    df.drop(columns)
    df = rename_cols(df)
    df = cast_types(df)
    df = change_member_casual(df)

    df.write_parquet(filename)

For the second set of date ranges, we'll only need to cast types for each column. We'll copy and modify the cast_types function to handle this.

In [24]:
'''
Function to cast types for each column
'''
def cast_types_2(df):
    out = df.select(
        pl.col('ride_id').cast(pl.Utf8, strict=False),
        pl.col('rideable_type').cast(pl.Utf8, strict=False),
        pl.col('started_at').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S', strict=False),
        pl.col('ended_at').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S', strict=False),
        pl.col('start_station_id').cast(pl.Int32, strict=False),
        pl.col('start_station_name').cast(pl.Utf8, strict=False),
        pl.col('start_lat').cast(pl.Float32, strict=False),
        pl.col('start_lng').cast(pl.Float32, strict=False),
        pl.col('end_station_id').cast(pl.Int32, strict=False),
        pl.col('end_station_name').cast(pl.Utf8, strict=False),
        pl.col('end_lat').cast(pl.Float32, strict=False),
        pl.col('end_lng').cast(pl.Float32, strict=False),
        pl.col('member_casual').cast(pl.Utf8, strict=False),
    )
    return out

In [25]:
for date_range in date_ranges_2:
    filename = date_range[0][:4] + '.parquet'
    df = create_df(date_range, datapath)
    df = cast_types_2(df)

    df.write_parquet(filename)

### Decisions made while processing the data

After processing the data, I decided against joining all the data for two reasons:
1. The data would be too large to process on my machine. 
    - Even with the reduced file size (4.93GB vs 36.79GB), a single dataframe would be cumbersome to work with.
2. The data is not consistent across the two date ranges.
    - The station ID is different
    - bike_id is not available for the second date range
    - rideable_type and ride_id is not available for the first date range
 
 I did not want to drop bike_id and rideable_type because they are interesting datapoints to explore. 

#### bike_id
Looking at the data from 2013 - 2021, I noticed that the bike_id might be a interesting datapoint to look at. You could potentially trace the history of a bike and see how many times it was used, how long it was used for, and where it was used. From 2021 to 2023, bike_id is replaced by ride_id. We are not able to create the same analysis for ride_id because it is a unique identified for each ride.

#### rideable_type
Rideable_type was added in 2021, with the values being: "classic_bike", "electric_bike" and "docked_bike". Although E-bikes were added to the Citi Bike fleet in 2018, the datapoint is not available until 2021. 
 

We have three options:
1. Drop the column
2. Fill in the data (e.g. assume that all rides from 2013 - 2018 were on classic bikes, and all rides from 2018 - 2021 as mix)
3. Create a model to predict the rideable_type based on the other data points
4. Keep the data as-is
 
 
 We'll go with option 4 for now, and revisit option 3 in the future.

#### Saving the data
The data is saved as a parquet file, one for each year. 
 
Saving as a parquet file significantly reduces the file size and allows for faster processing. By having a separate parquet file for each year, we can keep a consistent approach in analyzing the data. 